In [1]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset


# Load a dataset from the Hugging Face Hub
dataset = load_dataset("sst2")

# Simulate the few-shot regime by sampling 8 examples per class
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=5)
eval_dataset = dataset["validation"]

num_classes=2
# Load a SetFit model from Hub
model = SetFitModel.from_pretrained(
    "thenlper/gte-small",
    use_differentiable_head=True,
    head_params={"out_features": num_classes},
)


c:\Users\Colin\Documents\aml\aml_final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 583/583 [00:00<?, ?B/s] 
c:\Users\Colin\Documents\aml\aml_final\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Colin\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as a

In [2]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=8,
    num_iterations=5, # The number of text pairs to generate for contrastive learning
    num_epochs=1, # The number of epochs to use for contrastive learning
    column_mapping={"sentence": "text", "label": "label"} # Map dataset columns to text/label expected by trainer
)

# Train and evaluate
trainer.freeze() # Freeze the head
trainer.train() # Train only the body

Applying column mapping to training dataset
Generating Training Pairs: 100%|██████████| 5/5 [00:00<?, ?it/s]
***** Running training *****
  Num examples = 100
  Num epochs = 1
  Total optimization steps = 13
  Total train batch size = 8
Epoch: 100%|██████████| 1/1 [00:18<00:00, 18.20s/it]


In [3]:
# Unfreeze the head and freeze the body -> head-only training
trainer.unfreeze(keep_body_frozen=True)
# or
# Unfreeze the head and unfreeze the body -> end-to-end training
trainer.unfreeze(keep_body_frozen=False)

trainer.train(
    num_epochs=10, # The number of epochs to train the head or the whole model (body and head)
    batch_size=8,
    body_learning_rate=1e-5, # The body's learning rate
    learning_rate=1e-2, # The head's learning rate
    l2_weight=0.0, # Weight decay on **both** the body and head. If `None`, will use 0.01.
)
metrics = trainer.evaluate()
metrics

Applying column mapping to training dataset
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.
Epoch: 100%|██████████| 10/10 [03:04<00:00, 18.50s/it]
Applying column mapping to evaluation dataset
***** Running evaluation *****


{'accuracy': 0.8096330275229358}

In [4]:
# Run inference
preds = model(["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"])
preds

tensor([0, 0])